In [1]:
import numpy as np
from scipy.ndimage import imread
import scipy
import glob
from PIL import Image
import random
import cv2
import matplotlib.pyplot as plt

In [2]:
def SIFT(img):
    sift = cv2.SIFT()
    keypoints = sift.detect(img,None)
    keypoints, descriptors = sift.compute(img,keypoints)
    out = cv2.drawKeypoints(img,keypoints)
    return out,keypoints,descriptors


def keypoint2Array(kp):
    kp1_array_x = []
    kp1_array_y = []
    
    for point in kp1:
        kp1_array_x.append(point.pt[0])
        kp1_array_y.append(point.pt[1])
    
    return np.vstack((kp1_array_x, kp1_array_y)).T

In [3]:
#Taken from http://stackoverflow.com/questions/20259025/module-object-has-no-attribute-drawmatches-opencv-python
#Not available for Python

def drawMatches(img1, kp1, img2, kp2, matches):
    
    # Create a new output image that concatenates the two images together
    # (a.k.a) a montage
    
    rows1 = img1.shape[0]
    cols1 = img1.shape[1]
    rows2 = img2.shape[0]
    cols2 = img2.shape[1]

    out = np.zeros((max([rows1,rows2]),cols1+cols2,3), dtype='uint8')

    # Place the first image to the left
    out[:rows1,:cols1] = np.dstack([img1, img1, img1])

    # Place the next image to the right of it
    out[:rows2,cols1:] = np.dstack([img2, img2, img2])

    # For each pair of points we have between both images
    # draw circles, then connect a line between them
    for mat in matches:

        # Get the matching keypoints for each of the images
        img1_idx = mat.queryIdx
        img2_idx = mat.trainIdx

        # x - columns
        # y - rows
        (x1,y1) = kp1[img1_idx].pt
        (x2,y2) = kp2[img2_idx].pt

        # Draw a small circle at both co-ordinates
        # radius 4
        # colour blue
        # thickness = 1
        cv2.circle(out, (int(x1),int(y1)), 4, (255, 0, 0), 1)   
        cv2.circle(out, (int(x2)+cols1,int(y2)), 4, (255, 0, 0), 1)

        # Draw a line in between the two points
        # thickness = 1
        # colour blue
        cv2.line(out, (int(x1),int(y1)), (int(x2)+cols1,int(y2)), (255, 0, 0), 3)


    # Show the image
    plt.imshow(out, cmap = 'gray', interpolation = 'bicubic')
    plt.show()
    
    """
    #Resolution is pretty big
    cv2.imshow('Matched Features', out)
    cv2.waitKey(0)
    cv2.destroyWindow('Matched Features')
    """

    # Also return the image if you'd like a copy
    return out

In [4]:
# 0 means GrayScale
img1 = cv2.imread('PGPShelf/20150616_074411.jpg',0)
img2 = cv2.imread('PGPShelf/20150616_074444.jpg',0)
#cv2.imshow('Original GrayScale', img1)
#cv2.waitKey()
#plt.imshow(img1, cmap = 'gray', interpolation = 'bicubic')
sift1,kp1,desc1 = SIFT(img1)
kp_array_1 = keypoint2Array(kp1)

sift2,kp2,desc2 = SIFT(img2)
kp_array_2 = keypoint2Array(kp2)

In [36]:
#NORM_L2 means euclidean distance
matcher = cv2.BFMatcher(cv2.NORM_L2)
matching_points = matcher.knnMatch(desc1,desc2,k=2)
#best_matches = sorted(matching_points, key = lambda x:x.distance)
#matcher = cv2.BFMatcher(cv2.NORM_L2)
#best_matches = matcher.knnMatch(desc1,desc2,k=10)
#print len(best_matches)
# Sort them in the order of their distance Likelihood .

In [69]:
#Selecting good matching points
good = []
print "Total Correspondances" , len(matching_points)

i=0
for m,n in matching_points:
    i = i+1
    
    if abs(m.distance - n.distance) > 150:
        good.append(m)
        
print "Number of good correspondences", len(good)

Total Correspondances 26158
Number of good correspondences 35


In [68]:
#Taking correspondences points

point1 = []
point2 = []

for mat in good:
        # Get the matching keypoints for each of the images
        img1_idx = mat.queryIdx
        img2_idx = mat.trainIdx

        # x - columns
        # y - rows
        (x1,y1) = kp1[img1_idx].pt
        point1.append([x1,y1])
        (x2,y2) = kp2[img2_idx].pt
        point2.append([x2,y2])
        
assert len(point1) == len(point2)

point1 = np.array(point1)
point2 = np.array(point2)
print "Done"

Done


In [110]:
img3 = drawMatches(img1,kp1,img2,kp2,good)

In [60]:
# Calibration Matrix and 
# General Projective Camera (According Slide 24  03-CamerasPGP
M,mask = cv2.findHomography(point1,point2)
#QR Descomposition

R,K = scipy.linalg.qr(M)
K = K /K[2,2]

fix_t = np.eye(3);
if K[0,0] < 0:
    fix_t[0,0] = -1
if K[1,1] < 0:
    fix_t[1,1] = -1

K = np.dot(K , fix_t)

R = np.dot(fix_t , R)

if np.linalg.det(R) <0:
    R = -R ;

    
print K
s = K[0,1]
ax = K[0,0]
ay = K[1,1]
s = K[0,1]
x0 = K[0,2]
y0 = K[1,2]

T = M[2,:2]

[[  1.17083582e+00   1.13130372e-01  -6.86583869e+02]
 [  0.00000000e+00   1.03386276e+00   8.83346636e+01]
 [  0.00000000e+00   0.00000000e+00   1.00000000e+00]]


In [113]:
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*7,3), np.float32)
#print "objp: ",objp
objp[:,:2] = np.mgrid[0:7,0:6].T.reshape(-1,2)
#print "objp: ",objp
# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

#images = glob.glob('PGPShelf/*.jpg')
images=[]
images.append('PGPShelf/20150616_074411.jpg')
images.append('PGPShelf/20150616_074444.jpg')

points = []
points.append(point1)
points.append(point2)

i = 0
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chess board corners
    ret = True
    corners = points[i]
    i = i+1
    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
        #corners2 = cv2.cornerSubPix(img1,corners,(3,3),(-1,-1),criteria)
        imgpoints.append(corners)

        # Draw and display the corners
        #cv2.drawChessboardCorners(img, (7,6), corners,ret)
        #cv2.imshow('img',img)
        #cv2.waitKey(500)

#cv2.destroyAllWindows()
print img1.shape[::-1]
print len(objpoints)
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img1.shape[::-1],None,None) 


(3264, 2448)
2


error: /home/travis/miniconda/conda-bld/work/opencv-2.4.11/modules/calib3d/src/calibration.cpp:3193: error: (-215) ni > 0 && ni == ni1 in function collectCalibrationData


In [109]:
K_ = K[:2,:2]

p2_cal = np.dot(K_,point1.T).T
print point2 - p2_cal

[[-211.53213073  170.13947525]
 [  97.64076973   47.75142478]
 [-167.10037518   20.991513  ]
 [  14.7807308  -105.04989923]
 [-505.48909684  122.83365345]
 [ 111.65562906   75.84556569]
 [ 136.53936052 -134.57455554]
 [ 308.12024859   36.12945694]
 [-562.30917247  103.66196759]
 [-333.95980567  -75.71522578]
 [  82.30929534 -134.39069223]
 [-432.73703908 -222.82156519]
 [ 196.22275757 -133.58293907]
 [-481.42033658 -235.38462789]
 [ 222.44487546   98.45740076]
 [-210.47139173 -171.64265227]
 [ 135.28554485 -136.06769872]
 [-486.17169043 -237.53728111]
 [ 194.98160626  136.73121703]
 [ 238.82992927  125.37124543]
 [-197.49487881  -98.79583543]
 [ 267.09297734  -99.32774134]
 [ 139.33324394 -132.9545549 ]
 [ 252.49283193   72.98091468]
 [-370.00281905 -125.24230345]
 [-312.67571999 -131.31733217]
 [ 271.83802183  -97.72122979]
 [-333.40282872  -89.48390928]
 [-305.6936364  -130.80061962]
 [ 501.62617745   64.52397124]
 [ 130.38925773   95.08541038]
 [ 160.8758389    89.15763989]
 [-417.4

In [112]:
dist = np.array([0.00000000000000,0.00000000000,0,0])
img_undistorted = cv2.undistort(img2,K,dist)
#plt.imshow(img1, cmap = 'gray', interpolation = 'bicubic')
#plt.show()
plt.imshow(img_undistorted, cmap = 'gray', interpolation = 'bicubic')
plt.show()

In [65]:
cv2.cornerSubPix?